# Headway Explorer Tool

This tool is for exploring headway data given your specifications. You must specify desired service days, time period, and maximum headway length (in minutes). The resulting table flags the Bay Area transit service that meet the criteria you specified.


### TODO:

- add visualization of stops
- add route type (e.g. bus) and buffer selector tool

In [ ]:
from widget_fns import *
from headway_fns import (pull_GTFS_tables,
                         calc_headways,
                         get_headway_calc_period_str)

# I. Get data:

## Pull static GTFS tables for headway calculation

In [ ]:
# pull necessary GTFS tables from Redshift
df_dict = pull_GTFS_tables()

# II. Get user input:

## Specify desired service days:

In [ ]:
day_type_button, weekday_checkboxes = init_day_type_button()
display(day_type_button)

In [ ]:
# save user-specified input (from above cells)
service_days = get_days(day_type_button, weekday_checkboxes)

## Specify desired time period:

In [ ]:
# make dropdown
time_period_menu, custom_time_dict = init_time_period_menu()
display(time_period_menu)

In [ ]:
start_time, end_time = get_time_period(time_period_menu, custom_time_dict)

## Specify desired maximum headway length (in minutes):

In [ ]:
headway_selector, custom_headway = init_headway_selector()
display(headway_selector)

In [ ]:
max_headway = get_headway(headway_selector, custom_headway)

# III. Compute headways table

In [ ]:
user_input_args = {'max_headway': max_headway,
                   'service_days': service_days,
                   'time_period': {'start_time_td': start_time,
                                   'end_time_td': end_time}}

headways_args = {**df_dict, **user_input_args}

calc_id = get_headway_calc_period_str(service_days, start_time, end_time)
headways_table = calc_headways(**headways_args)

# IV. Download and inspect data

In [ ]:
show_download_button(headways_table, calc_id)

In [ ]:
# displayed table must have pagination and filtering 
headways_table.head()

## TODO: show route/stops visualization

- for mapping, add stops (join with route_pattern_id) (stop id, lat, long, Route_Pattern_ID -> draw routes on map)